# Text Classification With Bert

## Import Libraries

In [1]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# import tensorflow.keras as K
# from PIL import Image
# from tensorflow.keras import layers
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.layers import Conv2D,Activation,MaxPool2D,Dropout,Dense,Flatten,Input,BatchNormalization
# from tensorflow.keras.models import Sequential,load_model,Model
# from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
# from matplotlib import pyplot as plt
# from IPython.display import clear_output
# import h5py
# from tensorflow.keras.callbacks import ModelCheckpoint
# import warnings
# warnings.filterwarnings('ignore','FutureWarnings')

import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
# import tensorflow_text as text
# from official.nlp import optimization  # to create AdamW optimizer
from transformers import BertTokenizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [2]:
# print(tf.__version__) # 2.4.0
# print(K.__version__) # 2.4.0

In [3]:
label_map = {
    0 : 'aeroplane', 1 : 'bicycle', 2 : 'bird', 3 : 'boat', 
    4 : 'bus', 5 : 'car', 6 : 'cat', 7 : 'chair', 8 : 'cow',
    9 : 'diningtable', 10 : 'dog', 11 : 'horse', 12 : 'motorbike',
    13 : 'person', 14 : 'pottedplant', 15 : 'sheep', 16 : 'sofa',
    17 : 'train', 18 : 'tvmonitor'
  }

In [4]:
CATEGORIES = list(label_map.values())

In [5]:
TRAIN_DATA_DIR = "./dataset/train/sentences/"
TEST_DATA_DIR = "./dataset/test/sentences/"
CATEGORIES = ["aeroplane", "bicycle", "bird", "boat", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", 
              "motorbike", "person", "pottedplant", "sheep","sofa", "train", "tvmonitor"]

In [6]:
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    TRAIN_DATA_DIR,
    label_mode="categorical",
    validation_split=0.2,
    subset='training',
    seed= 1337 
)

Found 532 files belonging to 19 classes.
Using 426 files for training.


In [7]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    TRAIN_DATA_DIR,
    label_mode="categorical",
    validation_split=0.2,
    subset='validation',
    seed= 1337 
)

Found 532 files belonging to 19 classes.
Using 106 files for validation.


In [8]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    TEST_DATA_DIR,
    label_mode="categorical"
)

Found 380 files belonging to 19 classes.


In [11]:
max_features = None
embedding_dim = 128
sequence_length = 500

In [12]:
vectorization_layer = \
        tf.keras.layers.experimental.preprocessing.TextVectorization(
#             max_tokens = max_features,
            output_mode = "int",
            output_sequence_length = sequence_length,
        )

In [13]:
text_ds = raw_train_ds.map(lambda x, y: x)
vectorization_layer.adapt(text_ds)

In [14]:
max_features = vectorization_layer.vocabulary_size()

In [15]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorization_layer(text), label

In [16]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [9]:
MODEL_NAME = 'bert-base-uncased'

In [21]:
from transformers import TFBertForSequenceClassification
learning_rate = 2e-5
epochs = 10

model = TFBertForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels= len(CATEGORIES),
        proxies={'http': 'foo.bar:3128', 'http://hostname': 'foo.bar:4012'}
    )
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.CategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
# tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=True)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_test_encoded)
bert_history = model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/10
